In [1]:
import pandas as pd

In [2]:
from data_extraction.dummy_data_extractor import extract_dummy_data

In [4]:
master_table = extract_dummy_data("dummy_data")

C:\Users\syounas\OneDrive - Enova Facilities Management\Tasks\GitHub\HubgradeDataCleaning\Sana\data_extraction\dummy_data_extractor.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2023-03-12 01:05:00+04:00' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  pythonDF.loc[i, 'dqStart'] = pd.to_datetime(df['ts'].iloc[i], format="%Y-%m-%dT%H:%M:%S%z Dubai")


In [6]:
print(master_table.head())

                                             pointID unit dqType  \
0  @p:dmc_All:r:2ddf07d5-ef59ca94 DMC Building 1 ...   °C  Nulls   
1  @p:dmc_All:r:2ddf07d5-ef59ca94 DMC Building 1 ...   °C  Nulls   

                     dqStart      dqDuration   pointInterval  \
0  2023-03-12 01:05:00+04:00 1 days 11:10:00 0 days 00:05:00   
1  2023-03-19 01:10:00+04:00 0 days 23:30:00 0 days 00:05:00   

                          features  \
0  [p:dmc_All:r:2de337c0-72b69972]   
1  [p:dmc_All:r:2de337c0-72b69972]   

                                                 his  
0                                  DMC Building 1...  
1                                  DMC Building 1...  


In [8]:
his_data = master_table['his']

In [10]:
print(his_data.head())

0                                    DMC Building 1...
1                                    DMC Building 1...
Name: his, dtype: object
